In [1]:
import context
import pandas as pd
import Fitting as Fit
import Gillespie as Glp

# initialise constants
species_lenser = ["x","y","z"]  # z inner und x free, y loosely bound
# loss side
L_lenser = pd.DataFrame({"reaction01":[0, 0, 0],  # außen - x
            "reaction02":[1, 0, 0],  # x - y
            "reaction03":[0, 1, 0],  # y - z
            "reaction11":[1, 0, 0],  # x - außen
            "reaction12":[0, 1, 0],  # y - x
            "reaction13":[0, 0, 1]},  # z - y
            index = species_lenser)
# gain side
R_lenser = pd.DataFrame({"reaction01":[1, 0, 0],  # außen - x
            "reaction02":[0, 1, 0],  # x - y
            "reaction03":[0, 0, 1],  # y - z
            "reaction11":[0, 0, 0],  # x - außen
            "reaction12":[1, 0, 0],  # y - x
            "reaction13":[0, 1, 0]},  # z - y
            index = species_lenser)
N_lenser = R_lenser - L_lenser

"""rateConstants_lenser = {"reaction01": 1,
                  "reaction02": 1,
                  "reaction03": 1,
                  "reaction11": 0.1,
                  "reaction12": 1,
                  "reaction13": 1}"""

startQuantities_lenser = {"x":[0],"y":[0],"z":[0]}

time_limit_lenser = 2000.0
runs_lenser = 2

colours_lenser = {"x":"red", "y":"forestgreen", "z":"lightblue"}

# TODO: Kann ich die Daten so einlesen, dass der Index gleich die Zeiten sind und daraus trotzdem noch Intervalle machen?
measurement = pd.read_csv("./../Daten/FRAP_comparison_all_new_AssemblyDynamicsOfPMLNuclearBodiesInLivingCells_cleaned.csv")
measured_times = measurement["time[s]"].tolist()
#print(measured_times)
time_intervals = Fit.divide_time_axis_equidistantly(measured_times)
#print(time_intervals)

def fitness(parameters):
    # simulate one or more Gillespie runs
    trajectory = Glp.monte_carlo_gillespie(parameters, L_lenser, N_lenser, startQuantities_lenser, runs=runs_lenser, time_max=time_limit_lenser)
    # list of tuples of gillespie times_list and added_quantities_list for output signal
    list_of_output_data = Glp.make_output_signal(trajectory, species_lenser)
    # DataFrame of averaged gillespie data per time interval, time intervals as index
    assigned_simulation_data = Fit.assign_simulation_times_to_time_ranges_average(time_intervals,measured_times,list_of_output_data)
    # Normieren (auf feste Anzahl in ROI bspw. 200)
    assigned_simulation_data = assigned_simulation_data/200
    #quality_of_fitness = evaluation(measurement, assigned_simulation_data)
    return assigned_simulation_data#quality_of_fitness

ret = fitness({"reaction01": 1,
             "reaction02": 1,
             "reaction03": 1,
             "reaction11": 0.1,
             "reaction12": 1,
             "reaction13": 1})


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [2]:
species = "PML I WT"
#print(measurement[species].to_frame().reindex(index=time_intervals, fill_value=1))
print(measurement)
print(ret)

    time[s]  PML I WT  PML I -/-  PML II WT  PML II -/- mobil  \
0       -20  1.000000   1.000000   1.000000          1.000000   
1         0  0.093235   0.012242   0.064443          0.066914   
2        20  0.158290   0.022431   0.151296          0.103358   
3        40  0.212454   0.033544   0.230771          0.134008   
4        60  0.262379   0.029490   0.293022          0.150271   
5        80  0.295263   0.037560   0.324238          0.186385   
6       100  0.316648   0.046285   0.358991          0.205567   
7       120  0.359054   0.047522   0.382050          0.213984   
8       140  0.395668   0.043847   0.437840          0.230775   
9       160  0.407605   0.055869   0.443735          0.246495   
10      180  0.436885   0.056762   0.478782          0.265690   
11      200  0.458531   0.058291   0.471002          0.272568   
12      220  0.480783   0.061487   0.516955          0.261250   
13      240  0.507066   0.064528   0.520292          0.284983   
14      260  0.522567   0